In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Preprocesamiento inicial de la tabla de datos de seguimiento nutricional**

####**Descripción:** 

Abre el archivo original de tomas nutricionales `Tomas.zip` por fragmentos y le asigna a cada columna el nombre y el tipo de datos de Pandas (*Pandas datatypes*) más adecuado, con el fin de  ahorrar memoria y facilitar su posterior unión y procesamiento. Extrae y normaliza la información correspondiente a las tomas de datos del seguimiento nutricional, generando una tabla de tomas nutricionales (una fila por toma nutricional) separada de los datos propios del beneficiario. Guarda estas tablas de datos en formato binario `parquet` para ahorrar memoria.

####**Requerimientos:**

- Diccionario `datatypes_dictionaries/dtypes_tomas.json`, que mapea cada columna de la tabla de datos de tomas nutricionales al *datatype* de *Pandas* más adecuado.

- Diccionario `datatypes_dictionaries/dtypes_vars_toma.json`, que mapea cada *variable* asociada a una toma nutricional (sin el prefijo que indica la toma) al *datatype* de *Pandas* más adecuado.

-  Archivo `datatypes_dictionaries/cols_tomas.txt` con los nombres asignados a las columnas del conjunto de datos de monitoreo nutricional.

-  Archivo `raw_data/Tomas.zip` original, con la tabla de datos nutricionales en formato `csv` (separador |, codificación `utf-16le`), sumistrada por el ICBF bajo el nombre [` ICBF_Reincidenciadesnutricion_Tomasnutricionales_2017_2019.csv`](https://www.dropbox.com/s/s3tzmy7dvfi1mfu/Johanna%20Lozano%20Lavao%20-%20ICBF_reinicidenciadesnutricion.zip).

####**Salidas:**

- 100 tablas de datos `data_chunks/tomas_beneficiario/tomas_beneficiario_i.parquet` (`i` = 1, ..., 100) con las variables asociadas a cada beneficiario de cada fragmento (primeras 19 columnas, hasta la columna 18).

- 100 tablas de datos `data_chunks/tomas_normalizado/tomas_normalizado_i.parquet` (`i` = 1, ..., 100) con las variables asociadas a cada toma nutricional de cada fragmento (columnas 19 en adelante).

- Tabla de datos `preprocessed_data/tomas_beneficiario.parquet` con las variables asociadas a cada beneficiario (~6 millones de filas, una por cada fila en la tabla original).

- Tabla de datos `preprocessed_data/tomas_normalizado.parquet` con las variables asociadas a cada toma nutricional (~19 millones de filas, una por cada toma nutricional).

- Archivo `log_files/log_tomas_preprocesamiento.txt` con el reporte de cambios realizados.

In [ ]:
# Librerias relevantes
import pandas as pd
import json
import numpy as np
import time
from utils import replace_log

### **Preliminares**
Define algunos diccionarios de reemplazo útiles y algunos subconjuntos de columnas útiles a partir de los diccionarios de tipos de datos.

Define la partición que se hará de las filas de la tabla de datos de tomas nutricionales.


In [ ]:
# Cuenta el número de filas en el archivo
! zcat 'raw_data/Tomas.zip' | wc -l


gzip: raw_data/Tomas.zip: invalid compressed data--length error
6007562


In [ ]:
# Reemplazo de datos para asegurar su correcto procesamiento
# Reemplaza ',' por '.' en columnas de valores decimales
reemplazo_float = {',': '.'}
# Reemplaza 'M ' y 'f' por 'F', 'H ' por 'M' e 'I ' por 'I' en la columna 'Sexo'
reemplazo_sexo = {'M ': 'F',
                  'f':  'F',
                  'H ': 'M', 
                  'I ': 'I'}
# Corrige ortografía en las columnas de estado nutricional
reemplazo_estado = {'nutrici.n': 'nutrición',
                    ' Sobrepeso': ' sobrepeso',
                    ' Aguda': ' aguda',
                    ' Moderada': ' moderada'}
# Reemplaza las fechas absurdas (año > 2800) por su equivalente más probable                     
reemplazo_fecha = {'^2[89]': '20'}
 
# Abre los diccionarios con la estructura de datos definida
with open('datatypes_dictionaries/dtypes_tomas.json', 'r') as file:
  dtypes_tom = json.load(file)
with open('datatypes_dictionaries/dtypes_vars_toma.json', 'r') as file:
  dtypes_vars_toma = json.load(file)
 
# Nombres de las columnas
with open('datatypes_dictionaries/cols_tomas.txt', 'r') as file:
  cols_tom = ['Registro'] + [line.strip() for line in file.readlines()]
# La tabla original no incluye la variable 'IdToma'
cols_var_toma = list(dtypes_vars_toma.keys())[1:]
# Columnas de identificadores: 'Registro', 'Vigencia', 'Id', 'IdBeneficiario'
cols_id = cols_tom[:4]
# Columnas propias del beneficiario
cols_beneficiario = cols_tom[:19]
# Columnas propias de cada toma nutricional
cols_tomas = cols_id + cols_tom[19:]
# Columnas propias de las tomas nutricionales de la Dirección de Primera Infancia
cols_DPI = cols_id + cols_tom[19:99]
# Columnas propias de las tomas nutricionales de la Dirección de Nutrición
cols_DN =  cols_id + cols_tom[99:]
 
# Columnas de variables tipo flotante
cols_float = [col for col in cols_tom if str(dtypes_tom[col]) == 'float32']
# Columnas de la variable 'Toma': 'Toma1'... 'Toma12N'
cols_toma = [col for col in cols_tom if col[:4] == 'Toma']
# Columnas de variables tipo fecha
cols_fecha = [col for col in cols_tom if col[:5] == 'Fecha']
# Columnas de variables de estado nutricional: 'Estado...T1'... 'Estado...T12N'
# ('EstadoTallaEdad', 'EstadoPesoEdad',  'EstadoPesoTalla',  'EstadoIMC')
cols_estado = [col for col in cols_tom if col[:6] == 'Estado']

# Número total de registros, de fragmentos y número de registros en cada fragmento
n_registros = 6007562
n = 100
registros_fragmento = n_registros // n + 1
# Lista de números de fila de inicio y finalización de cada fragmento
# (El primer registro está en la fila 0, el último registro está en la fila 6007561)
particion = [registros_fragmento * i for i in range(n)] + [n_registros]

# Lista de reportes de cambios realizados
log = list()

### **Preprocesamiento del archivo por fragmentos**

Toma el archivo `Tomas.zip`, de 863 MB, comprimido del archivo original llamado [` ICBF_Reincidenciadesnutricion_Tomasnutricionales_2017_2019.csv`](https://www.dropbox.com/s/s3tzmy7dvfi1mfu/Johanna%20Lozano%20Lavao%20-%20ICBF_reinicidenciadesnutricion.zip) (originalmente comprimido en formato `rar`, tal como se descargó del repositorio de Dropbox enviado a nosotros por Correlation One el 16 de junio de 2021),  codificado en formato `utf-16le`, del directorio `raw_data` y lo subdivide en `n` = 100 fragmentos, de ~ 60076 registros cada uno. Requiere el diccionario de datos del archivo `dtypes_tomas.json` con la estructura de datos predefinida.

#### **1. Lectura**
Lee todas las columnas columnas de cada fragmento, desde la fila que indica `particion[i]` hasta la fila que indica `particion[i + 1] - 1`, en un dataframe llamado `tom`.

#### **2. Preprocesamiento inicial**
Preprocesa este dataframe:
- Reemplaza la `'H '` de *hombre* por `'M'` de *masculino* y la `'M '` de *mujer* por `'F'` de *femenino* en la variable `Sexo`.
- Reemplaza comas por puntos en las columnas que según el diccionario de datos deben ser de valores flotantes.
- Corrige algunos errores ortográficos en las variables de `Estado`: Tilde en *desnutrición*, solo mayúscula inicial (formato oración).
- Reemplaza algunas fechas absurdas (año > 2800) encontradas para que puedan ser transformadas en tipo `datetime64`.
- Asigna el tipo adecuado a cada variable para mejorar el manejo de memoria.

#### **3. Segmentación en dataframes complementarios**

- **Datos por beneficiario**: Extrae las primeras columnas, asociadas al beneficiario en un dataframe llamado `tom_beneficiario`.

- **Datos de la Dirección de Primera Infancia**: Si existen, extrae los valores de las columnas asociadas a las tomas nutricionales `Toma1`, ..., `Toma4` (realizadas por la Dirección de Primera Infancia) a otro dataframe llamado `tom_DPI`.

- **Datos de la Dirección de Nutrición**: Si existen, extrae los valores de las columnas asociadas a las tomas nutricionales `Toma1N`, ..., `Toma12N` (realizadas por la Dirección de Nutrición) a otro dataframe llamado `tom_DN`.

#### **4. Normalización de las tomas nutricionales**
Los valores de las tomas nutricionales tomadas por ambas direcciones (`tom_DPI` y `tom_DN`) se apilan por tomas agregando un identificador por cada toma `IdToma` y el registro de la `Direccion` de donde provienen y se concatenan en un dataframe llamado `tom_melted`.

#### **5. Guardado**
Guarda los dataframes preprocesados en un formato binario comprimido `parquet`, que permite guardar la estructura de los tipos de datos, en el directorio `data_chunks`.
- Guarda el dataframe `tom_beneficiario` completo en el archivo `tomas_beneficiario_i.parquet` (`i` = 1, ..., 100) del directorio `data_chunks/tomas_beneficiario`.
- Guarda el dataframe `tom_melted` completo en el archivo `tomas_normalizado_i.parquet` (`i` = 1, ..., 100) del directorio `data_chunks/tomas_normalizado`.

In [ ]:
t0 = time.time()
# Para cada fragmento definido que empiece en particion[i] a particion[i + 1] - 1:
for i in range(n):
  # 1. Lee el fragmento completo
  # La tabla original no incluye la variable 'Registro'
  tom = pd.read_csv('raw_data/Tomas.zip', sep = '|', encoding = 'utf-16le', header = 0,
                    skiprows = range(particion[i]), nrows = particion[i + 1] - particion[i], 
                    names=cols_tom[1:])
  print(f'Fragmento {i + 1}/{n} leído: {time.time() - t0:.2f} s.')
  
  # 2. Preprocesamiento inicial del dataframe
  tom = tom.reset_index().rename(columns={'index': 'Registro'})
  # Asigna un número de registro (fila) adecuado, según la tabla original
  tom['Registro'] = pd.Series(range(particion[i], particion[i + 1]))
  # Realiza los reemplazos de valores descritos
  # Corrige ',' por '.' en columnas de valores decimales que no hayan sido leídas como tales
  for col in cols_float:
    if str(tom[col].dtypes)[:5] != 'float':
      tom[col] = tom[col].replace(reemplazo_float, regex = True)
  # Corrige las fechas absurdas en columnas de tipo fecha que no estén vacías
  for col in cols_fecha:
    if tom[col].notnull().sum() > 0:
      tom[col] = replace_log(tom, col, col_id='Registro', map=reemplazo_fecha, log=log)
  tom['Sexo'] = tom['Sexo'].replace(reemplazo_sexo, regex = False)
  tom[cols_estado] = tom[cols_estado].replace(reemplazo_estado, regex = True)
  # Asigna a las variables categóricas codificadas por enteros, el tipo de datos entero, 
  # para prevenir que se guarden como flotantes
  tom[cols_toma] = tom[cols_toma].astype('UInt16')
  tom['Vigencia'] = tom['Vigencia'].astype('UInt16')
  tom['CodigoPaisNacimiento'] = tom['CodigoPaisNacimiento'].astype('UInt16')
  # Asignar a cada variable según su tipo de acuerdo al diccionario de datos
  for col in cols_tom:
    tom[col] = tom[col].astype(dtypes_tom[col])
  print(f'Fragmento {i + 1}/{n} preprocesado: {time.time() - t0:.2f} s.')
  
  # 3. Segmentación en dataframes complementarios
  # Extraer los datos del beneficiario
  tom_beneficiario = tom[cols_beneficiario]
  # Extraer los registros que tienen alguna toma de cada dirección
  # (D. de Primera Infancia y D. de Nutrición)
  existen_tomas_DN = tom[cols_DN[4:]].notna().sum(axis='columns') > 0
  existen_tomas_DPI = tom[cols_DPI[4:]].notna().sum(axis='columns') > 0
  tom_DN = tom[cols_DN][existen_tomas_DN].reset_index(drop=True)
  tom_DPI = tom[cols_DPI][existen_tomas_DPI].reset_index(drop=True)

  # 4. Normalización de las tomas nutricionales
  # Apilamiento (melt) guardando el identificador de la fila
  melted_DPI = tom_DPI.melt(id_vars = cols_id)
  # A cada [variable, valor] le asigna la toma según el sufijo de variable
  melted_DPI['toma'] = melted_DPI['variable'].str.extract(r'(\d+N*$)')
  # A cada [variable, valor] le asigna el nombre de variable apropiado (sin sufijos)
  melted_DPI['variable'] = melted_DPI['variable'].str.replace(r'T*\d+N*', '', regex=True)
  # Desapilamiento (pivot) para que los valores del mismo registro y toma se ubiquen en la misma fila
  pivot_DPI = melted_DPI.pivot(index=cols_id + ['toma'], columns='variable', values='value')
  # Remueve los valores que tienen un valor faltante en 'Toma' (toma nutricional faltante)
  pivot_DPI = pivot_DPI.dropna(subset=['Toma']).reset_index(drop=False)
  pivot_DPI['Toma'] = pivot_DPI['toma']
  # Asigna a todas las filas 'AntecedentePremadurez': '-' y 'Direccion': 'Primera Infancia'
  pivot_DPI['AntecedentePremadurez'] = pd.Series(['-'] * len(pivot_DPI))
  pivot_DPI['Direccion'] = pd.Series(['Primera Infancia'] * len(pivot_DPI))
  # Reordena las columnas
  pivot_DPI = pivot_DPI[cols_var_toma]
  if len(tom_DN) > 0:
    # Apilamiento (melt) guardando el identificador de la fila
    melted_DN = tom_DN.melt(id_vars = cols_id)
    # A cada [variable, valor] le asigna la toma según el sufijo de variable
    melted_DN['toma'] = melted_DN['variable'].str.extract(r'(\d+N*$)')
    # A cada [variable, valor] le asigna el nombre de variable apropiado (sin sufijos)
    melted_DN['variable'] = melted_DN['variable'].str.replace(r'T*\d+N*', '', regex=True)
    # Desapilamiento (pivot) para que los valores del mismo registro y toma se ubiquen en la misma fila
    pivot_DN = melted_DN.pivot(index=cols_id + ['toma'], columns='variable', values='value')
    # Remueve los valores que tienen un valor faltante en 'Toma' (toma nutricional faltante)
    pivot_DN = pivot_DN.dropna(subset=['Toma']).reset_index(drop=False)
    pivot_DN['Toma'] = pivot_DN['toma']
    # Asigna a todas las filas 'Direccion': 'Nutrición'
    pivot_DN['Direccion'] = pd.Series(['Nutrición'] * len(pivot_DN))
    # Reordena las columnas
    pivot_DN = pivot_DN[cols_var_toma]
    # Concatena la información de las tomas de ambas direcciones
    tom_melted = pivot_DPI.append(pivot_DN)
  else:
    tom_melted = pivot_DPI
  # Asigna cada columna al tipo de dato que le corresponde
  for col in tom_melted.columns:
    tom_melted[col] = tom_melted[col].astype(dtypes_vars_toma[col])
  print(f'Fragmento {i + 1}/{n} normalizado: {time.time() - t0:.2f} s.')
  
  # 5. Guardado de archivos
  # Guardar los dataframes preprocesado en un archivo .parquet
  tom_beneficiario.to_parquet(f'data_chunks/tomas_beneficiario/tomas_beneficiario_{i + 1}.parquet')
  tom_melted.to_parquet(f'data_chunks/tomas_normalizado/tomas_normalizado_{i + 1}.parquet')
  # Guarda el dataframe con tomas de la Direccion de Nutrición solo si hay algún registro
  print(f'Fragmento {i + 1}/{n} guardado: {time.time() - t0:.2f} s.')  
  # Elimina los dataframes temporales
  del tom
  del tom_beneficiario
  del tom_DPI
  del tom_DN
  del tom_melted

Fragmento 1/100 leído: 5.06 s.
Fragmento 1/100 preprocesado: 22.11 s.
Fragmento 1/100 normalizado: 47.85 s.
Fragmento 1/100 guardado: 50.80 s.
Fragmento 2/100 leído: 54.23 s.
Fragmento 2/100 preprocesado: 70.96 s.
Fragmento 2/100 normalizado: 96.95 s.
Fragmento 2/100 guardado: 97.77 s.
Fragmento 3/100 leído: 102.12 s.
Fragmento 3/100 preprocesado: 118.94 s.
Fragmento 3/100 normalizado: 144.35 s.
Fragmento 3/100 guardado: 145.17 s.
Fragmento 4/100 leído: 150.45 s.
Fragmento 4/100 preprocesado: 167.33 s.
Fragmento 4/100 normalizado: 193.11 s.
Fragmento 4/100 guardado: 193.87 s.
Fragmento 5/100 leído: 199.98 s.
Fragmento 5/100 preprocesado: 217.40 s.
Fragmento 5/100 normalizado: 243.51 s.
Fragmento 5/100 guardado: 244.51 s.
Fragmento 6/100 leído: 251.57 s.
Fragmento 6/100 preprocesado: 268.61 s.
Fragmento 6/100 normalizado: 294.10 s.
Fragmento 6/100 guardado: 294.87 s.


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fragmento 7/100 leído: 302.90 s.
Fragmento 7/100 preprocesado: 320.22 s.
Fragmento 7/100 normalizado: 346.14 s.
Fragmento 7/100 guardado: 346.98 s.


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,13,14,98,100,102,103,104,105,106,107,108,109,110,111,112,113,115,116,118,120,121,123,124,125,126,127,128,129,130,131,132,133,134,136,137,140,141,143,144,147,151,152,153,154,156,157) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fragmento 8/100 leído: 356.03 s.
Fragmento 8/100 preprocesado: 373.95 s.
Fragmento 8/100 normalizado: 399.41 s.
Fragmento 8/100 guardado: 400.29 s.


...

Fragmento 85/100 leído: 7789.63 s.
Fragmento 85/100 preprocesado: 7810.31 s.
Fragmento 85/100 normalizado: 7839.42 s.
Fragmento 85/100 guardado: 7840.80 s.


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (103,104,105,109,124,125,126,130,144,145,146,150,164,165,166,170,183,184,185,189,200,201,203,204,205,206,207,208,209,210,211,212,213,215,216,218,220,221,223,224,225,226,227,228,229,230,231,232,233,235,236,238,240,241,243,244,245,246,247,248,249,250,251,252,253,255,256,258,260,261,263,264,265,266,267,268,269,270,271,272,273,275,276,278,280,281,283,286,287,288,290,291,292,293,295,296,298,300,301,303,306,307,308,310,311,312,313,315,316,318) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fragmento 86/100 leído: 7928.61 s.
Fragmento 86/100 preprocesado: 7950.85 s.
Fragmento 86/100 normalizado: 7977.85 s.
Fragmento 86/100 guardado: 7978.80 s.


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,98,100,102,103,104,105,106,107,108,109,110,111,112,113,115,116,118,120,121,123,124,125,126,127,128,129,130,131,132,133,134,136,137,140,141,143,144,145,146,147,148,149,150,151,152,153,154,156,157,160,161,163,164,165,166,167,168,169,170,171,172,173,174,176,177,180,181,183,184,185,186,187,188,189,190,191,192,193,195,196,198,200,201,203,204,205,206,207,208,209,210,211,212,213,215,216,218,220,221,223,224,225,226,227,228,229,230,231,232,233,235,236,238,240,241,243,244,245,246,247,248,249,250,251,252,253,255,256,258,260,261,263,264,265,266,267,268,269,270,271,272,273,275,276,278,280,281,283,284,286,287,288,289,290,291,292,293,295,296,298) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fragmento 87/100 leído: 8065.54 s.
Fragmento 87/100 preprocesado: 8086.92 s.
Fragmento 87/100 normalizado: 8114.10 s.
Fragmento 87/100 guardado: 8114.95 s.
Fragmento 88/100 leído: 8201.99 s.
Fragmento 88/100 preprocesado: 8214.83 s.
Fragmento 88/100 normalizado: 8242.06 s.
Fragmento 88/100 guardado: 8242.96 s.
Fragmento 89/100 leído: 8330.98 s.
Fragmento 89/100 preprocesado: 8344.22 s.
Fragmento 89/100 normalizado: 8370.32 s.
Fragmento 89/100 guardado: 8371.17 s.
Fragmento 90/100 leído: 8459.74 s.
Fragmento 90/100 preprocesado: 8472.57 s.
Fragmento 90/100 normalizado: 8498.54 s.
Fragmento 90/100 guardado: 8499.48 s.
Fragmento 91/100 leído: 8592.91 s.
Fragmento 91/100 preprocesado: 8606.80 s.
Fragmento 91/100 normalizado: 8633.56 s.
Fragmento 91/100 guardado: 8634.40 s.
Fragmento 92/100 leído: 8725.77 s.
Fragmento 92/100 preprocesado: 8738.64 s.
Fragmento 92/100 normalizado: 8765.20 s.
Fragmento 92/100 guardado: 8766.06 s.
Fragmento 93/100 leído: 8858.12 s.
Fragmento 93/100 preprocesado

### **Unión de los fragmentos homólogos**
Une los `n` = 100 fragmentos preprocesados `tomas_beneficiario_i.parquet` y `tomas_normalizado_i.parquet` de los subdirectorios del directorio `data_chunks` en archivos únicos. Requiere el diccionario de datos del archivo `tomas_datatypes.json` con la estructura de datos definida.

####**Inicialización**
Define un dataframe vacío (llamado `tom_beneficiario` o `tom_normalizado`), a partir de los nombres apropiados para sus columnas.

####**Lectura**
Lee cada uno de los fragmentos guardados en archivos (`tomas_beneficiario_i.parquet` o `tomas_normalizado_i.parquet`, con `i` = 1, ..., 100) en un dataframe llamado `tom`.

#### **Unión**
Concatena el dataframe `tom` al dataframe completo y lo borra.

#### **Preprocesamiento**
Una vez concatenados todos los fragmentos:
- Asigna el tipo adecuado a cada variable.
- Reinicia los índices para evitar repeticiones.

#### **Guardado**
Guarda el dataframe completo (`tom_beneficiario` o `tom_normalizado`)en el archivo de formato binario comprimido (`tomas_beneficiario.parquet` o `tomas_normalizado.parquet`) en el directorio `preprocessed_data`.

In [ ]:
t0 = time.time()
print('\nUnión de los fragmentos con datos del beneficiario')
tom_beneficiario = pd.DataFrame(columns = cols_beneficiario)
# Toma cada archivo de tomas normalizado
for i in range(n):
  # Lee cada archivo en un dataframe y lo concatena al dataframe completo
  tom = pd.read_parquet(f'data_chunks/tomas_beneficiario/tomas_beneficiario_{i + 1}.parquet')
  tom_beneficiario = tom_beneficiario.append(tom)
  del tom
  if (i + 1) % 10 ==0:
    print(f'Partes {i + 1}/{n} unidas: {time.time() - t0:.2f} s.')
# Evita la repetición de índices
tom_beneficiario = tom_beneficiario.reset_index(drop = True)
for col in tom_beneficiario.columns:
  tom_beneficiario[col] = tom_beneficiario[col].astype(dtypes_tom[col])
# Guarda el dataframe a un nuevo archivo .parquet
tom_beneficiario.to_parquet('preprocessed_data/tomas_beneficiario.parquet')
print(f'Archivo listo: {time.time() - t0:.2f} s.')


Unión de los fragmentos con datos del beneficiario
Partes 10/100 unidas: 1.61 s.
Partes 20/100 unidas: 4.50 s.
Partes 30/100 unidas: 8.82 s.
Partes 40/100 unidas: 14.62 s.
Partes 50/100 unidas: 22.13 s.
Partes 60/100 unidas: 31.00 s.
Partes 70/100 unidas: 41.40 s.
Partes 80/100 unidas: 53.21 s.
Partes 90/100 unidas: 66.26 s.
Partes 100/100 unidas: 80.59 s.
Archivo listo: 88.60 s.


In [ ]:
t0 = time.time()
print('\nUnión de los fragmentos normalizados')
tom_normalizado = pd.DataFrame(columns = cols_var_toma)
# Toma cada archivo de tomas normalizado
for i in range(n):
  # Lee cada archivo en un dataframe y lo concatena al dataframe completo
  tom = pd.read_parquet(f'data_chunks/tomas_normalizado/tomas_normalizado_{i + 1}.parquet')
  tom_normalizado = tom_normalizado.append(tom)
  del tom
  if (i + 1) % 10 == 0:
    print(f'Partes {i + 1}/{n} unidas: {time.time() - t0:.2f} s.')
# Ordena las tomas por 'IdBeneficiario' y fecha de la toma
tom_normalizado = tom_normalizado.sort_values(['IdBeneficiario', 'FechaValoracionNutricional', 'Toma'])
# Genera un consecutivo de la toma nutricional 'IdToma'
tom_normalizado = tom_normalizado.reset_index(drop=True).reset_index().rename(columns={'index': 'IdToma'})
for col in tom_normalizado.columns:
  tom_normalizado[col] = tom_normalizado[col].astype(dtypes_vars_toma[col])
# Guarda el dataframe a un nuevo archivo .parquet
tom_normalizado.to_parquet('preprocessed_data/tomas_normalizado.parquet')
print(f'Archivo listo: {time.time() - t0:.2f} s.')


Unión de los fragmentos normalizados
Partes 10/100 unidas: 4.95 s.
Partes 20/100 unidas: 18.37 s.
Partes 30/100 unidas: 40.53 s.
Partes 40/100 unidas: 69.33 s.
Partes 50/100 unidas: 106.18 s.
Partes 60/100 unidas: 151.28 s.
Partes 70/100 unidas: 209.15 s.
Partes 80/100 unidas: 279.43 s.
Partes 90/100 unidas: 358.08 s.
Partes 100/100 unidas: 456.30 s.
Archivo listo: 643.27 s.


### **Reporte de cambios**

In [ ]:
t0 = time.time()
print('\nReporte de cambios')
# Escribe un archivo con los cambios generados
with open('log_files/log_tomas_preprocesamiento.txt', 'w') as log_file:
  log_file.write('Preprocesamiento_Tomas:\n')
  for line in log:
    log_file.write('\t' + line + '\n')
print(f'Archivo listo: {time.time() - t0:.2f} s.')


Reporte de cambios
Archivo listo: 0.25 s.
